In [272]:
# copied from

In [2]:
from metrics import get_baseline_data
import numpy as np
import math
import tensorflow
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import RobustScaler

2022-03-04 23:04:11.772227: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-04 23:04:11.774078: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [11]:
data = get_baseline_data("raw_data/preproc_data_rate.csv")
data.head()

,Date,Anchieta,Bangu,Barra da Tijuca,Botafogo,Campo Grande,Centro,Cidade de Deus,Complexo do Alemao,Copacabana,...,Portuaria,Ramos,Realengo,Rio Comprido,Rocinha,Santa Cruz,Santa Teresa,Sao Cristovao,Tijuca,Vila Isabel
0,2008-12-31,0.025522,0.009079,0.011007,0.000000,0.010674,0.066617,0.000000,0.0,0.034691,...,0.000000,0.006877,0.026449,0.015969,0.000000,0.002863,0.057947,0.052674,0.006297,0.000000
1,2009-01-01,0.063703,0.047398,0.052289,0.057072,0.066402,0.103458,0.198661,0.0,0.070387,...,0.029703,0.062168,0.046459,0.114169,0.014499,0.062111,0.000000,0.072225,0.012807,0.053311
2,2009-01-02,0.031852,0.015799,0.010458,0.009512,0.045433,0.206917,0.000000,0.0,0.056310,...,0.000000,0.034538,0.025811,0.016310,0.000000,0.028232,0.192864,0.018056,0.025613,0.038079
3,2009-01-03,0.006370,0.040626,0.017430,0.014268,0.027959,0.103458,0.198661,0.0,0.042232,...,0.059406,0.041445,0.041297,0.048929,0.014499,0.014116,0.096432,0.054169,0.019210,0.015232
4,2009-01-04,0.006370,0.029341,0.006972,0.014268,0.033201,0.206917,0.099331,0.0,0.035194,...,0.000000,0.034538,0.036135,0.114169,0.000000,0.042349,0.000000,0.126394,0.012807,0.007616


In [3]:
## Notes for group members:
# Attention: This model is fitted with a regular validation set. We should actually adjust it in a way that the validation
# set consists of a number of randomly selected sequences.
# On March 4, this model was also uploaded to Google Drive.
# To get the model from Google drive, see the corresponding link in the slack group.

data = get_baseline_data("raw_data/preproc_data_rate.csv")
data_wo_date = data.drop(columns="Date")

def subsample_sequence(data, length): # Return a shorter dataframe with specified length
    last_possible = data.shape[0] - length
    random_start = np.random.randint(0, last_possible)
    data_sample = data[random_start: random_start+length]
    return data_sample

def split_subsample_sequence(data, length): # Return a random sequence of specified length
    data_subsample = subsample_sequence(data, length)
    y_sample = data_subsample.iloc[length-31:]

    X_sample = data_subsample[0:length-31]
    X_sample = X_sample.values
    return np.array(X_sample), np.array(y_sample)

def get_X_y(data, n_sequences, length): # Return a sepcific number of (X,y) samples of specified length for all adm. regions

    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(data, length)
        X.append(xi)
        y.append(yi)

    X = np.array(X)
    y = np.array(y)
    return X, y

def get_train_test(data,n_sequences,length): # Return train and test data

    len_ = int(0.8*data.shape[0])
    data_train = data[:len_]
    data_test = data[len_:]

    test_seq = math.floor(n_sequences/4)

    X_train, y_train = get_X_y(data_train, n_sequences, length)
    X_test, y_test = get_X_y(data_test, test_seq, length)

#     X_train = X_train.reshape(X_train.shape[0], X_train.shape[1],1)
#     X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],1)

    return X_train, y_train, X_test, y_test

def model(number_of_sequences, input_sequence_length, number_of_regions, prediction_horizon):
# number_of_sequences = 1000 # number of data sequences for training
# input_sequence_length = 169 # sequence length in training process
# number_of_regions = 30 # number of region sin training process
# prediction_horizon = 31 # number of predicted days by region
#X_train_shape = (number_of_sequences, input_sequence_length, number_of_regions)
#y_train_shape = (number_of_sequences, input_sequence_length, number_of_regions)

    model = models.Sequential()
    model.add(layers.LSTM(40, return_sequences=False, activation="tanh", input_shape = (input_sequence_length, number_of_regions)))
    model.add(layers.RepeatVector(prediction_horizon))
    model.add(layers.LSTM(40, return_sequences=True, activation="tanh"))
    model.add(layers.TimeDistributed(layers.Dense(number_of_regions,"relu")))
    model.compile(loss="mse",
                optimizer="rmsprop")
    return model

def fit_model(X_train, y_train, model):
    es = EarlyStopping(monitor='val_loss', verbose=1, patience=20, restore_best_weights=True)
    hist = model.fit(X_train, y_train, callbacks=[es],epochs = 2000,validation_split =0.3, batch_size=32)
    return hist


In [4]:
data = get_baseline_data("raw_data/preproc_data_rate.csv")
data_wo_date = data.drop(columns="Date")

In [5]:
_,_, X_test, y_test = get_train_test(data,1000,200)

In [9]:
X_test

array([[[Timestamp('2017-11-13 00:00:00'), 0.0126335843626546,
         0.0498241424398579, ..., 0.1803125718432903,
         0.0807233990715341, 0.0629027523448748],
        [Timestamp('2017-11-14 00:00:00'), 0.0568511296319457,
         0.0411590741894478, ..., 0.2028516433237016,
         0.0660464174221643, 0.0489243629349026],
        [Timestamp('2017-11-15 00:00:00'), 0.063167921813273,
         0.0476578753772554, ..., 0.1126953574020564,
         0.0220154724740547, 0.0559135576398887],
        ...,
        [Timestamp('2018-04-28 00:00:00'), 0.0441972764375479,
         0.0345042196504118, ..., 0.3014520714395032,
         0.1419211038175283, 0.063946341913985],
        [Timestamp('2018-04-29 00:00:00'), 0.0694528629732897,
         0.0474433020193163, ..., 0.1623203461597325,
         0.0522867224590893, 0.0852617892186467],
        [Timestamp('2018-04-30 00:00:00'), 0.0505111730714834,
         0.066851925572673, ..., 0.324640692319465, 0.0746953177986991,
         0.09947208

In [7]:
from tensorflow import keras
model_AK = keras.models.load_model('RNN_LSTM')
model_AK.summary()
model_AK.predict(X_test)

2022-03-04 23:04:54.292648: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-04 23:04:54.294913: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-04 23:04:54.295014: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-RMC00DV9): /proc/driver/nvidia/version does not exist
2022-03-04 23:04:54.313446: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential_65"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_110 (LSTM)             (None, 40)                11360     
                                                                 
 repeat_vector_25 (RepeatVec  (None, 31, 40)           0         
 tor)                                                            
                                                                 
 lstm_111 (LSTM)             (None, 31, 40)            12960     
                                                                 
 time_distributed_66 (TimeDi  (None, 31, 30)           1230      
 stributed)                                                      
                                                                 
Total params: 25,550
Trainable params: 25,550
Non-trainable params: 0
_________________________________________________________________


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Timestamp).